In [5]:
tables = camelot.read_pdf('./Group 1/epd4115.pdf', pages='8' ,process_background=True)
print(len(tables))

5


In [11]:
tables = camelot.read_pdf('./Group 2/epd4923.pdf', pages='all')
print(len(tables))

9


In [19]:
tables[4].df

,0,1,2,3,4,5,6
0,HWD\nkg,"9,19E-06","9,35E-12","5,60E-12","1,97E-12","6,34E-12","0,00E+00"
1,NHWD\nkg,"1,47E-02","2,51E-05","1,67E-05","9,47E-06","2,98E-01","0,00E+00"
2,RWD\nkg,"4,12E-04","5,27E-06","1,93E-07","4,56E-07","6,27E-07","0,00E+00"
3,Components for re-use\nkg,"0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"
4,Materials for recycling\nkg,"1,50E-02","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"
5,Materials for energy recovery\nkg,"0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"
6,"Exported energy, electricity\nMJ","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"
7,"Exported energy, thermal\nMJ","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"


In [4]:
tables = camelot.read_pdf('./Group 3/epd5054.pdf', pages='all', process_background=True)
print(len(tables))

9


In [32]:
tables[4].df[tables[4].df[0].str.contains('(^GWP - Fossil)|(^GWP-\nfossil$)')]

C:\Users\mirol\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,0,1,2,3,4,5,6,7,8,9,10
2,GWP-\nfossil,kg CO2 \neq.,"4,89E+00","1,63E-01","3,87E-01","5,44E+00",0,"1,63E-02","1,03E-03","2,14E-01","-1,50E+00"


In [21]:
'WDP*' in tables[4].df.values

True

In [29]:
tables[8].df

,0
0,Results per 1 kg of brass mixer
1,BIOGENIC CARBON CONTENT \nUnit \nQUANTITY
2,Biogenic carbon content in product \nkg C \n0
3,Biogenic carbon content in packaging \nkg C \n...


In [34]:

tables = camelot.read_pdf('./Group 4/epd5190.pdf', pages='all')
print(len(tables))

16


In [3]:
import camelot
import pandas as pd
import openpyxl
import os
import pdfplumber
import re
from datetime import datetime
# tables = camelot.read_pdf('epd1388_2021.pdf', pages='10',process_background=True)
# tables = camelot.read_pdf('./IBU-09-20/New/14632.pdf', pages='6', split_text=True,table_areas=["69.60548391889084,500.088593765987,525.3099336281931,403.96614296894364"])
# print(len(tables))

# tables = camelot.read_pdf('./New/14699.pdf', pages='all')
# print(len(tables))

In [2]:
columns = {
    "impactGWP100_kgCO2e_total": '\(kg CO2 eq.\)\nGWPTOTAL',
    "impactGWP100_kgCO2e_fossil": '\(kg CO2 eq.\)\nGWPFOSSIL',
    "impactGWP100_kgCO2e_biogenic": '\(kg CO2 eq.\)\nGWPBIOGENIC',
    "impactGWP100_kgCO2e_luluc": '\(kg CO2 eq.\)\nGWPLULUC',
    "impactODP_WMO2014_kgCFC11e": 'ODP\n\(kg CFC 11 eq.\)',
    "impactAP_molHeq": 'AP\n\(mol H\+ eq\.\)',
    "impactEPFreshwater_kgPO4e": '\(kg \(PO4\)3- eq.\)\nEPFRESHWATER',
    "impactEPFreshwater_kgP": '\(kg P eq.\)\nEPFRESHWATER',
    "impactEPMarine_kgNe": '\(kg N eq.\)\nEPMARINE',
    "impactEPTerrestrial_molNe": '\(mol N eq.\)\nEPTERRESTRIAL',
    "impactPOCP_kgNMVOCe": 'POCP\n\(kg NMVOC eq.\)',
    
    "cleanWaterDeprivation_m3": '^WDP\*\n\(m3 world eq.\)',
    "particulateMatter_incidence": '^PM',
    "ionisingRadiation_kgU235eq": '^IR',
    "ecoToxicityFreshwater_CTUe": '^ETP - FW',
    "humanToxicityCancer_CTUh": '^HTTP - C',
    "humanToxicityNonCancer_CTUh": '^HTTP - NC',
    "potentialSoilQualityIndex": '^SQP',
  "impactGWP100_kgCO2e": "(^GWP - Total$)|(^Global warming potential \xad total$)|(^GWP-total$)|(^GWP$)|(^Global warming potential$)|(^Globales Erwärmungspotenzial ­ total$)|(^Globales Erwärmungspotenzial$)",
  "impactODP_kgCFC11e": "(^ODP)|((^Abbaupotenzial der stratosphärischen Ozonschicht$)|(^Depletion potential of the stratospheric ozone layer$)",
  "impactAP_kgSO2e": "(^AP)|(^AP Air$)|(^Versauerungspotenzial von Boden und Wasser$)|(^Acidification potential of land and water$)",
  "impactEP_kgPO4e": "(^Eutrophication, fraction of nutrients reaching freshwater \nend compartment$)|(^EP-freshwater$)|(^EP$)|(^Eutrophierungspotenzial$)|(^Eutrophication potential)",
  "impactPOCP_kgEthenee": "(^POCP$)|(^Bildungspotenzial für troposphärisches Ozon$)|(^Formation potential of tropospheric ozone photochemical \noxidants)",
  "impactADPElements_kgSbe": "(^\(kg Sb eq.\)\nADPMINERALS&METALS\*)|(^Abiotic depletion potential for non\xadfossil resources$)|(^ADPE$)|(^Potenzial für die Verknappung abiotischer Ressourcen \xad \nnicht fossile Ressourcen$)|(^Potenzial für die Verknappung abiotischer Ressourcen - \nnicht fossile Ressourcen$)|(^Abiotic depletion potential for non-fossil resources)",
  "impactADPFossilFuels_MJ": "(^\(MJ\)\nADPFOSSIL\*)|(^ADPF)|(^Potenzial für die Verknappung abiotischer Ressourcen \xad \nfossile Brennstoffe$)|(^Potenzial für die Verknappung abiotischer Ressourcen - \nfossile Brennstoffe$)|(^Abiotic depletion potential for fossil resources)",
    
  "renewablesUsedAsEnergy_MJ": "(^PERE)|(^Erneuerbare Primärenergie als Energieträger$)|(^Renewable primary energy as energy carrier$)",
  "renewablesUsedAsMaterial_MJ": "(^PERM)|(^Erneuerbare Primärenergie zur stofflichen Nutzung$)|(^Renewable primary energy resources as material utilization$)",
  "embodiedRenewableEnergy_MJ": "(^PERT)|(^Total erneuerbare Primärenergie$)|(^Total use of renewable primary energy resources$)",
  "nonRenewablesUsedAsEnergy_MJ": "(^PENRE)|(^Non\xadrenewable primary energy as energy carrier$)|(^Nicht-erneuerbare Primärenergie als Energieträger$)|(^Nicht\xaderneuerbare Primärenergie als Energieträger$)|(^Non-renewable primary energy as energy carrier$)",
  "nonRenewablesUsedAsMaterial_MJ": "(^PENRM)|(^Non\xadrenewable primary energy as material utilization$)|(^Nicht-erneuerbare Primärenergie zur stofflichen Nutzung$)|(^Nicht\xaderneuerbare Primärenergie zur stofflichen Nutzung$)|(^Non-renewable primary energy as material utilization$)",
  "embodiedEnergy_MJ": "(^PENRT)|(^Total nicht erneuerbare Primärenergie$)|(^Total use of non-renewable primary energy resources$)|(^Total use of non\xadrenewable primary energy resources$)",
  "recyclingMaterialUse_kg": "(^SM)|(^Einsatz von Sekundärstoffen$)|(^Use of secondary material$)",
  "renewableRecylingFuelUse_MJ": "(^RSF)|(^Erneuerbare Sekundärbrennstoffe$)|(^Use of renewable secondary fuels$)",
  "nonRenewableRecylingFuelUse_MJ": "(^NRSF)|(^Use of non\xadrenewable secondary fuels$)|(^Nicht-erneuerbare Sekundärbrennstoffe$)|(^Nicht\xaderneuerbare Sekundärbrennstoffe$)|(^Use of non-renewable secondary fuels$)",
  "cleanWaterNetUse_m3": "(^FW)|(^Nettoeinsatz von Süßwasserressourcen$)|(^Use of net fresh water$)",
    
  "wasteHazardous_kg": "(^HWD)|(^Gefährlicher Abfall zur Deponie$)|(^Hazardous waste disposed$)",
  "wasteNonHazardous_kg": "(^NHWD)|(^Non\xadhazardous waste disposed$)|(^Entsorgter nicht gefährlicher Abfall$)|(^Non-hazardous waste disposed$)",
  "wasteRadioactive_kg": "(^RWD)|(^Entsorgter radioaktiver Abfall$)|(^Radioactive waste disposed$)",
  "reusableMaterialsOutput_kg": "(^CRU)|(^Komponenten für die Wiederverwendung$)|(^Components for re-use$)|(^Components for re\xaduse$)",
  "recyclableMaterialsOutput_kg": "(^MFR)|(^Stoffe zum Recycling$)|(^Materials for recycling$)",
  "energyMaterialsOutput_kg": "(^MER)|(^Stoffe für die Energierückgewinnung)|(^Materials for energy recovery$)",
  "exportedEnergyOutput_MJ_electric": "(^EEE)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)",
  "exportedEnergyOutput_MJ_thermal": "(^EET)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)",
  "exportedEnergyOutput_MJ": "(^EE)",
    
  "traciGWP_kgCO2e": 8,
  "traciODP_kgCFC11e": 12,
  "traciAP_kgSO2e": 6,
  "traciEP_kgNe": 7,
  "traciPOCP_kgO3e": 10,
  "traciNRPE_MJ": 17
}

headers = [
    "impactGWP100_kgCO2e_total",
    "impactGWP100_kgCO2e_fossil",
    "impactGWP100_kgCO2e_biogenic",    
    'impactGWP100_kgCO2e_luluc',
    'impactODP_WMO2014_kgCFC11e',
    'impactAP_molHeq',
    'impactEPFreshwater_kgPO4e',
    'impactEPFreshwater_kgP',
    'impactEPMarine_kgNe',
    'impactEPTerrestrial_molNe',
    'impactPOCP_kgNMVOCe',
    'cleanWaterDeprivation_m3',
    'particulateMatter_incidence',
    'ionisingRadiation_kgU235eq',
    'ecoToxicityFreshwater_CTUe',
    'humanToxicityCancer_CTUh',
    'humanToxicityNonCancer_CTUh',
    'potentialSoilQualityIndex',
    
    "impactODP_kgCFC11e",
    "impactAP_kgSO2e",
    "impactEP_kgPO4e",
    "impactPOCP_kgEthenee",
    "impactADPElements_kgSbe",
    "impactADPFossilFuels_MJ",
    
    "renewablesUsedAsEnergy_MJ",
    "renewablesUsedAsMaterial_MJ",
    "embodiedRenewableEnergy_MJ",
    "nonRenewablesUsedAsEnergy_MJ",
    "nonRenewablesUsedAsMaterial_MJ",
    "embodiedEnergy_MJ",
    
    "recyclingMaterialUse_kg",
    "renewableRecylingFuelUse_MJ",
    "nonRenewableRecylingFuelUse_MJ",
    "cleanWaterNetUse_m3",
    "wasteHazardous_kg",
    "wasteNonHazardous_kg",
    "wasteRadioactive_kg",
    "reusableMaterialsOutput_kg",
    "recyclableMaterialsOutput_kg",
    "energyMaterialsOutput_kg",
    "exportedEnergyOutput_MJ_thermal",
    "exportedEnergyOutput_MJ_electric",
    "exportedEnergyOutput_MJ",
    
    "traciGWP_kgCO2e",
    "traciODP_kgCFC11e",
    "traciAP_kgSO2e",
    "traciEP_kgNe",
    "traciPOCP_kgO3e",
    "traciNRPE_MJ"
]

In [3]:
def epd_number(file_name):
    with pdfplumber.open('./Group 2/' + file_name) as pdf:
        first_page = pdf.pages[0]
        text = first_page.extract_text()
        #print(text)
        #item = re.findall('EPD registration number.*', text)
        #try:
        #    r = item[0].split(' ')[3]
        #except:
        #    r = '' 
        item = re.findall('S-P-.*', text)
        
    return item[0]

In [4]:

def make_excel_file(files_data):
    book = openpyxl.Workbook()
    filename = 'Envirodec_' + datetime.now().strftime("%Y-%m-%d") + '.xlsx'
    book.save(filename)

    sheet = book['Sheet']
    sheet.cell(row=1, column=1).value = 'Filename'
    sheet.cell(row=1, column=2).value = 'EPD_Number'
    sheet.cell(row=1, column=3).value = 'DownloadLink'
    sheet.cell(row=1, column=4).value = 'Stages'
    
    count = 5
    for header in headers:
        sheet.cell(row=1, column=count).value = header
        count += 1
    
    current_row = 2
    for file_data in files_data:
        print(file_data["file_name"])
        print(file_data["stages"])
        print("*******************")
        df = file_data["dataFrame"]
        base_added = False
        
        for stage in file_data["stages"]:
            if not base_added:
                sheet.cell(row=current_row, column=1).value = file_data["file_name"]
                sheet.cell(row=current_row, column=2).value = epd_number(file_data["file_name"])
                sheet.cell(row=current_row, column=3).value = '/mnt/data/Downloads/2021-12-20/Envirodec/New/' + file_data["file_name"]
                base_added = True

            sheet.cell(row=current_row, column=4).value = stage
            header_counter = 0
            for header in headers:
                
#                 i = file_data["stages"].index(stage) + 2
#                 sheet.cell(row=current_row, column= 5 + header_counter).number_format = '0.000E+00'
#                 value = float(df[df[0].str.contains(columns2[header])][i].to_list()[0])
#                 #value = float(df[df.eq(columns[header]).any(1)][i].to_list()[0])
# #                 sheet.cell(row=current_row, column= 5 + header_counter).value = value
                try:
                    num_of_columns = len(df.columns)                    
                    i = file_data["stages"].index(stage) + (num_of_columns - len(file_data["stages"]))
                    sheet.cell(row=current_row, column= 5 + header_counter).number_format = '0.000E+00'
                    replaced = df[df[0].str.contains(columns[header])][i].to_list()[0].replace(',', '.')
                    if not replaced == 'IND':
                        try:
                            value = float(replaced)
                        except:
                            try:
                                value = float(replaced.replace('\xad', '-'))
                            except:
                                value = float(replaced.replace('\n', ''))
                    else:
                        value = replaced
                        
                    #value = float(df[df.eq(columns[header]).any(1)][i].to_list()[0])
                    sheet.cell(row=current_row, column= 5 + header_counter).value = value
                except:
                    print("**************************************ERRROR*****************************")
                    print(file_data["file_name"])
                    print(stage)
                    print(columns[header])
                header_counter= header_counter +1
            current_row = current_row + 1
        current_row = current_row + 1
            
    book.save(filename)
    book.close()

In [5]:
def clean_df(df):
    transformed = df.applymap(lambda x: x.split(' ') if ('E-' in x) or ('E+' in x) or (x == '') else x)
    for i in range(transformed.shape[0]):
        row = transformed.iloc[[i]]
        for index, element in row.iloc[0].items():
            if element == ['']:
                for index_s, search in row.iloc[0].items():
                    if (len(search) > 1) and ('[kg' not in search) and (type(search) == list):
                        if index < index_s:        
                            transformed.loc[i,index] = [search[0]]
                            row.iloc[0][index] = [search[0]]
                            del search[0]
                        else: 
                            transformed.loc[i,index] = [search[1]]                            
                            row.iloc[0][index] = [search[1]]
                            del search[1]
                        break
    return transformed.applymap(lambda x: x[0] if type(x) == list else x)



In [7]:
files = os.listdir('./Group 2')
# files = ['15230.pdf', '14983.pdf.pdf', '14956.pdf']
r = []
for file in files:
    print("************ FILE NAME: " + file + " ****************")
    tables = camelot.read_pdf('./Group 2/' + file, pages='all')
    dataFrames = []

    file_info = {}
    stages = []
    hwd_added = False
    gwp_added = False
    pere_added = False
    for table in tables:
        df = table.df
        stages_added = True
        #stages = ['A1', 'A2', 'A3', 'C1', 'C2', 'C3', 'C4', 'Total', 'D']
        #stages = ['A1-A3', 'A4', 'A1-A4']
        #stages = ['A1-A2', 'A3', 'A4', 'C1', 'C2', 'C3', 'C4', 'D']
        #stages = ['A1-A3', 'A4', 'A5', 'C1', 'C2', 'C3', 'C4', 'D']
        stages = ['A1-A3', 'C1', 'C2', 'C3', 'C4', 'D']
        gwp = ['GWP - Total', '(kg CO2 eq.)\nGWPTOTAL']
        for g in gwp:
            if (g in df.values) and ('TRACI 2.1 Impact Assessment' not in df.values):
                if not stages_added:
                    try:

                        a = list(df[df.eq('Unit').any(1)].values[0])
                        unit_index = a.index('Unit') + 1
                        stages = a[unit_index:]
                    except:
                        a = list(df[df.eq('Einheit').any(1)].values[0])
                        unit_index = a.index('Einheit') + 1
                        stages = a[unit_index:]
                    stages_added = True
                #cleaned_df = clean_df(df)
                #dataFrames.append(cleaned_df)
                print(1)
                dataFrames.append(df)
                gwp_added = True


        pere = ['PERE\nMJ' ]
        for p in pere:
            if p in df.values:
                #cleaned_df = clean_df(df)
                #dataFrames.append(cleaned_df)
                dataFrames.append(df)
                print(2)
                pere_added = True

        hwd = ['HWD\nkg']

        for h in hwd:
            if h in df.values:
                #cleaned_df = clean_df(df)
                #dataFrames.append(cleaned_df)
                dataFrames.append(df)
                print(3)
                hwd_added = True
                

        if hwd_added and gwp_added and pere_added:       
            result = pd.concat(dataFrames)

            file_info["dataFrame"] = result


            file_info["file_name"] = file
            file_info["stages"] = stages

            r.append(file_info)
            hwd_added = False
            gwp_added = False
            pere_adeed = False
    

make_excel_file(r)


#r

************ FILE NAME: epd4923.pdf ****************
1
2
3
************ FILE NAME: epd4924.pdf ****************
1
2
3
************ FILE NAME: epd4925.pdf ****************
1
2
3
************ FILE NAME: epd4926.pdf ****************
1
2
3
************ FILE NAME: epd4927.pdf ****************
1
2
3
************ FILE NAME: epd4928.pdf ****************
1
2
3
epd4923.pdf
['A1-A3', 'C1', 'C2', 'C3', 'C4', 'D']
*******************
**************************************ERRROR*****************************
epd4923.pdf
A1-A3
^PM
**************************************ERRROR*****************************
epd4923.pdf
A1-A3
^IR
**************************************ERRROR*****************************
epd4923.pdf
A1-A3
^ETP - FW
**************************************ERRROR*****************************
epd4923.pdf
A1-A3
^HTTP - C
**************************************ERRROR*****************************
epd4923.pdf
A1-A3
^HTTP - NC
**************************************ERRROR*****************************
ep

C:\Users\mirol\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)



C3
(^MFR)|(^Stoffe zum Recycling$)|(^Materials for recycling$)
**************************************ERRROR*****************************
epd4923.pdf
C3
(^MER)|(^Stoffe für die Energierückgewinnung)|(^Materials for energy recovery$)
**************************************ERRROR*****************************
epd4923.pdf
C3
(^EET)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)
**************************************ERRROR*****************************
epd4923.pdf
C3
(^EEE)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)
**************************************ERRROR*****************************
epd4923.pdf
C3
(^EE)
**************************************ERRROR*****************************
epd4923.pdf
C3
8
**************************************ERRROR*****************************
epd4923.pdf
C3
12
**************************************ERRROR*****************************
epd4923.pdf
C3
6
**************************************ERRROR*****************************
ep

C4
^SQP
**************************************ERRROR*****************************
epd4924.pdf
C4
(^ODP)|((^Abbaupotenzial der stratosphärischen Ozonschicht$)|(^Depletion potential of the stratospheric ozone layer$)
**************************************ERRROR*****************************
epd4924.pdf
C4
(^Eutrophication, fraction of nutrients reaching freshwater 
end compartment$)|(^EP-freshwater$)|(^EP$)|(^Eutrophierungspotenzial$)|(^Eutrophication potential)
**************************************ERRROR*****************************
epd4924.pdf
C4
(^POCP$)|(^Bildungspotenzial für troposphärisches Ozon$)|(^Formation potential of tropospheric ozone photochemical 
oxidants)
**************************************ERRROR*****************************
epd4924.pdf
C4
(^CRU)|(^Komponenten für die Wiederverwendung$)|(^Components for re-use$)|(^Components for re­use$)
**************************************ERRROR*****************************
epd4924.pdf
C4
(^MFR)|(^Stoffe zum Recycling$)|(^Materials

**************************************ERRROR*****************************
epd4925.pdf
C3
(^CRU)|(^Komponenten für die Wiederverwendung$)|(^Components for re-use$)|(^Components for re­use$)
**************************************ERRROR*****************************
epd4925.pdf
C3
(^MFR)|(^Stoffe zum Recycling$)|(^Materials for recycling$)
**************************************ERRROR*****************************
epd4925.pdf
C3
(^MER)|(^Stoffe für die Energierückgewinnung)|(^Materials for energy recovery$)
**************************************ERRROR*****************************
epd4925.pdf
C3
(^EET)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)
**************************************ERRROR*****************************
epd4925.pdf
C3
(^EEE)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)
**************************************ERRROR*****************************
epd4925.pdf
C3
(^EE)
**************************************ERRROR*****************************
ep

C4
(^MER)|(^Stoffe für die Energierückgewinnung)|(^Materials for energy recovery$)
**************************************ERRROR*****************************
epd4926.pdf
C4
(^EET)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)
**************************************ERRROR*****************************
epd4926.pdf
C4
(^EEE)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)
**************************************ERRROR*****************************
epd4926.pdf
C4
(^EE)
**************************************ERRROR*****************************
epd4926.pdf
C4
8
**************************************ERRROR*****************************
epd4926.pdf
C4
12
**************************************ERRROR*****************************
epd4926.pdf
C4
6
**************************************ERRROR*****************************
epd4926.pdf
C4
7
**************************************ERRROR*****************************
epd4926.pdf
C4
10
**************************************ERRRO

**************************************ERRROR*****************************
epd4927.pdf
C4
(^CRU)|(^Komponenten für die Wiederverwendung$)|(^Components for re-use$)|(^Components for re­use$)
**************************************ERRROR*****************************
epd4927.pdf
C4
(^MFR)|(^Stoffe zum Recycling$)|(^Materials for recycling$)
**************************************ERRROR*****************************
epd4927.pdf
C4
(^MER)|(^Stoffe für die Energierückgewinnung)|(^Materials for energy recovery$)
**************************************ERRROR*****************************
epd4927.pdf
C4
(^EET)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)
**************************************ERRROR*****************************
epd4927.pdf
C4
(^EEE)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)
**************************************ERRROR*****************************
epd4927.pdf
C4
(^EE)
**************************************ERRROR*****************************
ep

In [3]:
tables = camelot.read_pdf('./Group 1/epd1576.pdf', pages='all')
print(len(tables))

17


In [26]:
tables[4].df

,0,1,2,3,4,5,6,7,8
0,HWD\nkg,"4,70E-03","1,18E-10","5,69E-11","0,00E+00","2,20E-11","0,00E+00","5,55E-11","0,00E+00"
1,NHWD\nkg,"1,54E-02","3,65E-04","7,06E-02","0,00E+00","6,56E-05","0,00E+00","2,61E+00","0,00E+00"
2,RWD\nkg,"7,18E-04","3,04E-06","2,24E-05","0,00E+00","7,58E-07","0,00E+00","5,49E-06","0,00E+00"
3,Components for re-use\nkg,"0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"
4,Materials for recycling\nkg,"2,15E-01","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"
5,Materials for energy recovery\nkg,"0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"
6,"Exported energy, electricity\nMJ","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"
7,"Exported energy, thermal\nMJ","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"


In [7]:
'(kg CO2 eq.)\nGWPTOTAL' in tables[4].df.values

False

In [8]:
d = r[1]['dataFrame']

In [32]:

df[df[0].str.contains(columns['impactAP_kgSO2e'])][4].to_list()[0].replace(',', '.')

'0.00E+00'

In [33]:
df[df[0].str.contains(columns['impactGWP100_kgCO2e_total'])][4].to_list()[0].replace(',', '.')

'0.00E+00'

In [10]:
len(d[d[0].str.contains('^\(kg CO2 eq.\)\nGWPTOTAL')])

4

In [11]:
len(d)

128